In [1]:
import json
import numpy as np
import pandas as pd

In [2]:
cards = pd.read_json('cards.json')
cards.drop(columns=['description', 'arena', 'id'], inplace=True)
cards = cards.set_index('key')

In [3]:
cards

,name,sc_key,elixir,type,rarity
key,,,,,
knight,Knight,Knight,3,Troop,Common
archers,Archers,Archer,3,Troop,Common
goblins,Goblins,Goblins,2,Troop,Common
giant,Giant,Giant,5,Troop,Rare
pekka,P.E.K.K.A,Pekka,7,Troop,Epic
...,...,...,...,...,...
earthquake,Earthquake,Earthquake,3,Spell,Rare
barbarian-barrel,Barbarian Barrel,BarbLog,2,Spell,Epic
heal-spirit,Heal Spirit,Heal,1,Troop,Rare


In [4]:
cards.to_csv('data/cards.csv')

In [5]:
with open('cards_stats.json') as file:
    cards_stats_full = json.load(file)
print(cards_stats_full['characters'][0])

{'name': 'Knight', 'rarity': 'Common', 'flags': None, 'tribe': None, 'sight_range': 5500, 'deploy_time': 1000, 'charge_range': 0, 'speed': 60, 'hitpoints': 651, 'hit_speed': 1200, 'load_time': 700, 'damage': 79, 'damage_special': 0, 'crown_tower_damage_percent': 0, 'load_first_hit': False, 'load_after_retarget': False, 'stop_time_after_attack': 0, 'stop_time_after_special_attack': 0, 'projectile': None, 'custom_first_projectile': None, 'multiple_projectiles': 0, 'multiple_targets': 0, 'all_targets_hit': False, 'range': 1200, 'minimum_range': 0, 'special_min_range': 0, 'special_range': 0, 'special_load_time': 0, 'special_ignore_buildings': False, 'special_attacks_to_ignore_list': False, 'attacks_ground': True, 'attacks_air': False, 'death_damage_radius': 0, 'death_damage': 0, 'death_push_back': 0, 'attack_push_back': 0, 'melee_pushback': 0, 'life_time': 0, 'projectile_special': None, 'projectile_effect_special': None, 'area_damage_radius': 0, 'target_only_buildings': False, 'target_only

In [6]:
name_to_key = {}
for i in cards_stats_full:
    for j in cards_stats_full[i]:
        if 'key' in j:
            name_to_key[j['name']] = {j['key']}
            if 'summon_character' in j:
                if j['summon_character'] == 'Goblin':
                    print(j['name'], j['key'])
                if j['summon_character'] not in name_to_key:
                    name_to_key[j['summon_character']] = {j['key']}
                else:
                    name_to_key[j['summon_character']].add(j['key'])
                    
                if j['summon_character'] == 'Goblin':
                    print(name_to_key['Goblin'])

Goblins goblins
{'goblins'}
GoblinGang goblin-gang
{'goblin-gang', 'goblins'}


In [7]:
cards_by_key = {i: {} for i in cards.index.values}
cards_by_name = {}
for i in cards_stats_full:
#     print('---------------------------')
#     print(i)
#     print('------')
#     print(cards_stats_full[i][0].keys())
#     print('---------------------------')
    for j in cards_stats_full[i]:
        name = j['name']
        cards_by_name[name] = j
        
        if name in name_to_key:
#             print(name, name_to_key[name])
            for k in name_to_key[name]:
                if k is not None:
                    cards_by_key[k] = {**j, **cards_by_key[k]}

            
        if 'key' in j:
            if j['key'] in cards_by_key:
                cards_by_key[j['key']] = {**cards_by_key[j['key']] , **j}
            else:
                cards_by_key[j['key']] = j

In [8]:
cards_by_key['goblins']

{'name': 'Goblins',
 'rarity': 'Common',
 'flags': None,
 'tribe': 'Goblin',
 'sight_range': 5500,
 'deploy_time': 1000,
 'charge_range': 0,
 'speed': 120,
 'hitpoints': 79,
 'hit_speed': 1100,
 'load_time': 900,
 'damage': 47,
 'damage_special': 0,
 'crown_tower_damage_percent': 0,
 'load_first_hit': False,
 'load_after_retarget': False,
 'stop_time_after_attack': 0,
 'stop_time_after_special_attack': 0,
 'projectile': None,
 'custom_first_projectile': None,
 'multiple_projectiles': 0,
 'multiple_targets': 0,
 'all_targets_hit': False,
 'range': 500,
 'minimum_range': 0,
 'special_min_range': 0,
 'special_range': 0,
 'special_load_time': 0,
 'special_ignore_buildings': False,
 'special_attacks_to_ignore_list': False,
 'attacks_ground': True,
 'attacks_air': False,
 'death_damage_radius': 0,
 'death_damage': 0,
 'death_push_back': 0,
 'attack_push_back': 0,
 'melee_pushback': 0,
 'life_time': 0,
 'projectile_special': None,
 'projectile_effect_special': None,
 'area_damage_radius': 0,


In [9]:
detail = {}
missing = {}
for i in cards.index.values:
    if len(cards_by_key[i]) != 0:
        detail[i] = cards_by_key[i]
    else:
        detail[i] = None
        missing[i] = None
missing

{'elixir-collector': None,
 'fireball': None,
 'arrows': None,
 'rocket': None,
 'goblin-barrel': None,
 'mirror': None,
 'the-log': None,
 'barbarian-barrel': None,
 'giant-snowball': None}

In [10]:
missing['elixir-collector'] = 'ElixirCollector'
missing['fireball'] = 'FireballSpell'
missing['arrows'] = 'ArrowsSpell'
missing['rocket'] = 'RocketSpell'
missing['goblin-barrel'] = 'GoblinBarrelSpell'
missing['the-log'] = 'LogProjectile'
missing['giant-snowball'] = 'SnowballSpell'
missing['barbarian-barrel'] = 'BarbLogProjectile'

for i in missing:
    if missing[i] is not None:
        detail[i] = cards_by_name[missing[i]]

In [11]:
detail['knight']['attacks_ground']

True

In [12]:
def add_attribute(att):
    cards[att] = np.nan
    for i in cards.index.values:
        if detail[i] is not None and att in detail[i]:
            cards.loc[i, att] = detail[i][att]
    return cards

In [13]:
attributes = {'range', 'attacks_ground', 'attacks_air', 'flying_height', 'hits_ground', 'hits_air', 'aoe_to_ground', 'aoe_to_air'}
for i in attributes:
    add_attribute(i)

In [14]:
cards.fillna(False, inplace=True)
cards['range'] = cards['range'].astype(int)
cards['flying_height'] = cards['flying_height'].astype(int)
cards['damage_air'] = cards['hits_air'] | cards['attacks_air'] | cards['aoe_to_air']
cards['damage_ground'] = cards['hits_ground'] | cards['attacks_ground'] | cards['aoe_to_ground']
cards.drop(columns=['hits_air', 'attacks_air', 'aoe_to_air', 'hits_ground', 'attacks_ground', 'aoe_to_ground'], inplace=True)

In [15]:
cards

,name,sc_key,elixir,type,rarity,range,flying_height,damage_air,damage_ground
key,,,,,,,,,
knight,Knight,Knight,3,Troop,Common,1200,0,False,True
archers,Archers,Archer,3,Troop,Common,5000,0,True,True
goblins,Goblins,Goblins,2,Troop,Common,500,0,False,True
giant,Giant,Giant,5,Troop,Rare,1200,0,False,True
pekka,P.E.K.K.A,Pekka,7,Troop,Epic,1200,0,False,True
...,...,...,...,...,...,...,...,...,...
earthquake,Earthquake,Earthquake,3,Spell,Rare,0,0,False,True
barbarian-barrel,Barbarian Barrel,BarbLog,2,Spell,Epic,0,0,False,True
heal-spirit,Heal Spirit,Heal,1,Troop,Rare,0,0,True,True


In [16]:
cards.to_csv('data/cards.csv')